In [2]:
from matplotlib import cm
from matplotlib.colors import Normalize
import numpy as np

In [14]:
import torch

def compute_cd_dist(pred, gt):
    """Compute the Chamfer Distance borrowed from https://github.com/otaheri/chamfer_distance,
    you need to install the chamfer_distance package by using:

    pip install git+'https://github.com/otaheri/chamfer_distance'

    Args:
        pred (tensor): (B, N, 3)
        gt (tensor): (B, N, 3)

    Returns:
        tensor: scalar tensor
    """
    from chamfer_distance import ChamferDistance as chamfer_dist

    if not isinstance(pred, torch.Tensor):
        pred = torch.from_numpy(pred).float()
        gt = torch.from_numpy(gt).float()
    
    if len(pred.shape) == 2:
        pred = pred.unsqueeze(0)
        gt = gt.unsqueeze(0)
    print(pred.shape, gt.shape)    
    dist1, dist2, idx1, idx2 = chamfer_dist(pred, gt)
    dist_mean = (dist1 + dist2) / 2

    return dist_mean

In [27]:
pc_fp  = "/home/zhanghm/Research/Programming/cv-fighter/pc_utils/Neural_Pull_pred/Icosahedron_X4.xyz"
pc_fp = "/home/zhanghm/Research/Programming/cv-fighter/pc_utils/Neural_Pull_pred/elephant_X4.xyz"
pc_fp = "/home/zhanghm/Research/PU/Github/NeuralPull-Pytorch/data/dis_pu_output/Icosahedron_X4.xyz"
pc_arr = np.loadtxt(pc_fp)
print(pc_arr.shape)

pc_gt_fp = "/home/zhanghm/Research/Programming/cv-fighter/pc_utils/test/gt_FPS_8192_normalized/Icosahedron_X4.xyz"
# pc_gt_fp = "/home/zhanghm/Research/Programming/cv-fighter/pc_utils/test/gt_FPS_8192_normalized/elephant_X4.xyz"
pc_gt_arr = np.loadtxt(pc_gt_fp)
print(pc_gt_arr.shape)

error = compute_cd_dist(pc_arr, pc_gt_arr)[0]
print(error.shape)

cmap = cm.get_cmap('coolwarm')
# cmap = cm.get_cmap('rainbow')
norm = Normalize(vmin=error.min(), vmax=error.max())
rgba_values = cmap(norm(error))
print(rgba_values.shape)

(8192, 3)
(8192, 3)
torch.Size([1, 8192, 3]) torch.Size([1, 8192, 3])
torch.Size([8192])
(8192, 4)


In [29]:
import open3d as o3d

pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(pc_arr[:, :3])
pc.colors = o3d.utility.Vector3dVector(rgba_values[:, 0:3])
o3d.io.write_point_cloud(f'pc.ply', pc)

True